In [10]:
from networkx.utils.misc import flatten
from alt_reg import CMC_alt_reg
import numpy as np
import matplotlib.pyplot as plt
import generator as gen
import ridgeNN
from joblib import dump, load

# Design Studies

## 1. $\alpha$

### General MNAR

In [11]:
alphas = [0.01, 1, 10]
methods = ['PCR', 'Ridge', 'Lasso']

res_GMNAR = {}

N_sim = 5

for method in methods:
    res_GMNAR[method] = {}
    
    for alpha in alphas:
        res_GMNAR[method][str(alpha)] = {'MAE' : [], 'RMSE' : []}

for alpha in alphas:
    for _ in range(N_sim):   
        rating_matrix, P, latent_movie_matrix = gen.getRatingAndPropensityMatrix_general(inv_scale=1, alpha = alpha)
        D = np.random.binomial(1, P)
        Y = rating_matrix * D
        Y[D == 0] = np.nan

        params = {'reg_adap': 0.001,
                  'min_value': 1,
                  'max_value': 5,
                  'verbose': False}

        snn = CMC_alt_reg(**params)
        Y_PCR, Y_Ridge, Y_Lasso = snn.fit_transform(D, Y, fast = True)

        error = {'PCR' : (rating_matrix - Y_PCR).flatten(), 
                 'Ridge' : (rating_matrix - Y_Ridge).flatten(), 
                 'Lasso' : (rating_matrix - Y_Lasso).flatten()}

        for method in methods:
            res_GMNAR[method][str(alpha)]['MAE'].append(np.mean(np.abs(error[method])))
            res_GMNAR[method][str(alpha)]['RMSE'].append(np.sqrt(np.mean(error[method] ** 2)))
            
dump(res_GMNAR, './design_results/alphas_GMNAR.joblib')

['./design_results/alphas_GMNAR.joblib']

In [13]:
res_GMNAR = load('./design_results/alphas_GMNAR.joblib')

with open("./design_results/alphas_GMNAR.tex", "w") as f:
    
    f.write("\\begin{tabular}{l*{6}{c}} \n" +
            "\\toprule \n" +
            "& \\multicolumn{2}{c}{PCR} & \\multicolumn{2}{c}{Ridge} & \\multicolumn{2}{c}{LASSO} \\\\ \n" +
            "\\cmidrule(lr){2-3} \\cmidrule(lr){4-5} \\cmidrule(lr){6-7} \n" +
            "& MAE & RMSE & MAE & RMSE & MAE & RMSE \\\\ \n" + 
            "\\midrule \n \\addlinespace \n")
    
    f.write("\\addlinespace \n $\\alpha$ & \\multicolumn{6}{l}{\\textbf{Dirichlet Concentration Param.}} \\\\ \n \\addlinespace \n")
    for alpha in alphas:
        
        f.write(str(alpha) + " & ")
        
        f.write(" & ".join(["${:.3f} \\pm {:.3f}$ & ${:.3f} \\pm {:.3f}$".format(np.mean(res_GMNAR[method][str(alpha)]['MAE']),
                                                                           np.std(res_GMNAR[method][str(alpha)]['MAE'])/np.sqrt(N_sim),
                                                                           np.mean(res_GMNAR[method][str(alpha)]['RMSE']),
                                                                           np.std(res_GMNAR[method][str(alpha)]['RMSE'])/np.sqrt(N_sim)) for method in methods]))
        f.write(" \\\\ \n")

    f.write("\\bottomrule \n \\end{tabular}")

## 2. $r$

In [ ]:
rs = [2, 5, 20]
methods = ['PCR', 'Ridge', 'Lasso']

res_GMNAR = {}

N_sim = 5

for method in methods:
    res_GMNAR[method] = {}
    
    for r in rs:
        res_GMNAR[method][str(r)] = {'MAE' : [], 'RMSE' : []}

for r in rs:
    for _ in range(N_sim):   
        rating_matrix, P, latent_movie_matrix = gen.getRatingAndPropensityMatrix_general(inv_scale=1, r = r)
        D = np.random.binomial(1, P)
        Y = rating_matrix * D
        Y[D == 0] = np.nan

        params = {'reg_adap': 0.001,
                  'min_value': 1,
                  'max_value': 5,
                  'verbose': False}

        snn = CMC_alt_reg(**params)
        Y_PCR, Y_Ridge, Y_Lasso = snn.fit_transform(D, Y, fast = True)

        error = {'PCR' : (rating_matrix - Y_PCR).flatten(), 
                 'Ridge' : (rating_matrix - Y_Ridge).flatten(), 
                 'Lasso' : (rating_matrix - Y_Lasso).flatten()}

        for method in methods:
            res_GMNAR[method][str(r)]['MAE'].append(np.mean(np.abs(error[method])))
            res_GMNAR[method][str(r)]['RMSE'].append(np.sqrt(np.mean(error[method] ** 2)))
            
dump(res_GMNAR, './design_results/r_GMNAR.joblib')

In [ ]:
res_GMNAR = load('./design_results/r_GMNAR.joblib')

with open("./design_results/r_GMNAR.tex", "w") as f:
    
    f.write("\\begin{tabular}{l*{6}{c}} \n" +
            "\\toprule \n" +
            "& \\multicolumn{2}{c}{PCR} & \\multicolumn{2}{c}{Ridge} & \\multicolumn{2}{c}{LASSO} \\\\ \n" +
            "\\cmidrule(lr){2-3} \\cmidrule(lr){4-5} \\cmidrule(lr){6-7} \n" +
            "& MAE & RMSE & MAE & RMSE & MAE & RMSE \\\\ \n" + 
            "\\midrule \n \\addlinespace \n")
    
    f.write("\\addlinespace \n $r$ & \\multicolumn{6}{l}{\\textbf{Dimension of Latent Variables}} \\\\ \n \\addlinespace \n")
    for r in rs:
        
        f.write(str(r) + " & ")
        
        f.write(" & ".join(["${:.3f} \\pm {:.3f}$ & ${:.3f} \\pm {:.3f}$".format(np.mean(res_GMNAR[method][str(r)]['MAE']),
                                                                           np.std(res_GMNAR[method][str(r)]['MAE'])/np.sqrt(N_sim),
                                                                           np.mean(res_GMNAR[method][str(r)]['RMSE']),
                                                                           np.std(res_GMNAR[method][str(r)]['RMSE'])/np.sqrt(N_sim)) for method in methods]))
        f.write(" \\\\ \n")

    f.write("\\bottomrule \n \\end{tabular}")